<a href="https://colab.research.google.com/github/tarang1998/ML-AI-DL/blob/main/gen-ai/FineTunningOpenAIModelForClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

In [2]:
import os
import openai
import getpass

openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')

Enter OpenAI API key:··········


In [3]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

In [85]:
print(sports_dataset["data"][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [86]:
sports_dataset['target_names']

['rec.sport.baseball', 'rec.sport.hockey']

In [87]:
sports_dataset.target_names[sports_dataset['target'][0]]


'rec.sport.baseball'

In [88]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


In [89]:
import json
import random

# Load your original dataset (assumes sports_dataset.data and sports_dataset.target exist)
label_mapping = {0: "baseball", 1: "hockey"}
chat_data = []

# Convert dataset into OpenAI chat format
for text, label in zip(sports_dataset.data, sports_dataset.target):
    chat_entry = {
        "messages": [
            {"role": "system", "content": "Classify the following text into a sport category."},
            {"role": "user", "content": text},
            {"role": "assistant", "content": label_mapping[label]}
        ]
    }
    chat_data.append(chat_entry)

print(f"✅ Total Data Samples: {len(chat_data)}")


✅ Total Data Samples: 1197


In [91]:

# Validate each record
for i, entry in enumerate(chat_data):
    messages = entry.get("messages", [])
    if  not any(msg["role"] == "assistant" for msg in messages):
        print(f"❌ ERROR: Missing assistant message in entry {i}")

In [95]:
MAX_TOKENS = 500
def check_token_length(entry):
    text = " ".join(msg["content"] for msg in entry["messages"])
    return len(text.split())  # Approximate token count


filtered_chat_data=[]

# Check all entries
for i, entry in enumerate(chat_data):
    if check_token_length(entry) > MAX_TOKENS:
        print(f"❌ Entry {i} is too long and may be truncated.")
    else:
      filtered_chat_data.append(entry)

print(f"✅ Filtered Data Samples: {len(filtered_chat_data)}")



❌ Entry 7 is too long and may be truncated.
❌ Entry 11 is too long and may be truncated.
❌ Entry 18 is too long and may be truncated.
❌ Entry 26 is too long and may be truncated.
❌ Entry 32 is too long and may be truncated.
❌ Entry 42 is too long and may be truncated.
❌ Entry 45 is too long and may be truncated.
❌ Entry 51 is too long and may be truncated.
❌ Entry 71 is too long and may be truncated.
❌ Entry 74 is too long and may be truncated.
❌ Entry 75 is too long and may be truncated.
❌ Entry 78 is too long and may be truncated.
❌ Entry 84 is too long and may be truncated.
❌ Entry 98 is too long and may be truncated.
❌ Entry 100 is too long and may be truncated.
❌ Entry 105 is too long and may be truncated.
❌ Entry 106 is too long and may be truncated.
❌ Entry 114 is too long and may be truncated.
❌ Entry 138 is too long and may be truncated.
❌ Entry 139 is too long and may be truncated.
❌ Entry 151 is too long and may be truncated.
❌ Entry 153 is too long and may be truncated.
❌ E

In [96]:
# Shuffle the dataset for randomness
random.shuffle(filtered_chat_data)

# Define split sizes
train_size = int(0.8 * len(filtered_chat_data))  # 80% training, 20% validation

# Split dataset
train_data = filtered_chat_data[:train_size]
valid_data = filtered_chat_data[train_size:]

print(f"✅ Training Samples: {len(train_data)}, Validation Samples: {len(valid_data)}")

✅ Training Samples: 859, Validation Samples: 215


In [97]:
train_data[0]

{'messages': [{'role': 'system',
   'content': 'Classify the following text into a sport category.'},
  {'role': 'user',
   'content': "From: davewood@bruno.cs.colorado.edu (David Rex Wood)\nSubject: Re: Rockies need some relief\nNntp-Posting-Host: bruno.cs.colorado.edu\nOrganization: University of Colorado, Boulder\nLines: 14\n\nIn article <1993Apr18.225740.15978@colorado.edu> davewood@bruno.cs.colorado.edu (David Rex Wood) writes:\n>shaky) innings giving up just one run.  Then game the dreaded relief.  Three\n                                              ^^^^\n>picthers combined to give up 3 runs (one each I believe) in the 7th inning\n ^^^^^^^^\n>be taken lightly.  Going into today's game, the had the league's leading\n                                             ^^^\n\nGeez, can I type or what?\n\n-- \n-------------------------------------------------------------------------------\nDavid Rex Wood -- davewood@cs.colorado.edu -- University of Colorado at Boulder\n--------------------

In [98]:
# Save training data
with open("sports_train.jsonl", "w") as f:
    for entry in train_data:
        f.write(json.dumps(entry) + "\n")

# Save validation data
with open("sports_valid.jsonl", "w") as f:
    for entry in valid_data:
        f.write(json.dumps(entry) + "\n")

print("✅ Training and Validation datasets saved successfully!")

✅ Training and Validation datasets saved successfully!


In [99]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=openai.api_key)


# Upload training file
train_response = client.files.create(file=open("sports_train.jsonl", "rb"), purpose="fine-tune")
print(f"Training File ID: {train_response.id}")

# Upload validation file
valid_response = client.files.create(file=open("sports_valid.jsonl", "rb"), purpose="fine-tune")
print(f"Validation File ID: {valid_response.id}")

Training File ID: file-JTRf7oGaySMmkexkFwt529
Validation File ID: file-E2wEftoz6RSZ4eNir7ZoFn


In [100]:
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=train_response.id,
    validation_file=valid_response.id,
    model="gpt-3.5-turbo"
)

print(f"Fine-tuning Job ID: {fine_tune_job.id}")

Fine-tuning Job ID: ftjob-O3BR5V2ctGDvgIm5J39KnJ64


In [108]:
jobs = client.fine_tuning.jobs.list()
for job in jobs:
    print(f"Job ID: {job.id}, Status: {job.status}")

Job ID: ftjob-O3BR5V2ctGDvgIm5J39KnJ64, Status: succeeded
Job ID: ftjob-OYwJmvRTsj4kdkHK2Hg3W9ef, Status: failed
Job ID: ftjob-agbR5iLLi6BpV6XR162wUU9Y, Status: failed
Job ID: ftjob-bDQuY2uATS63pHm9KD2dr5ib, Status: failed
Job ID: ftjob-8fj9XgkKbCckt8piSpclKMLy, Status: failed


In [117]:
fine_tune_details = client.fine_tuning.jobs.retrieve("ftjob-O3BR5V2ctGDvgIm5J39KnJ64")
print(fine_tune_details)

FineTuningJob(id='ftjob-O3BR5V2ctGDvgIm5J39KnJ64', created_at=1738969899, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:tarangnair::AySrIowB', finished_at=1738973974, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-wPyaNuoepXOI7oF41dfodjab', result_files=['file-ER6LaR6g7rF6cQTgeJQ61h'], seed=803297784, status='succeeded', trained_tokens=895167, training_file='file-JTRf7oGaySMmkexkFwt529', validation_file='file-E2wEftoz6RSZ4eNir7ZoFn', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None)


In [111]:
# List all fine-tuned models
fine_tuned_models = [model.id for model in client.models.list() if "ft" in model.id]

# Display available fine-tuned models
if fine_tuned_models:
    print("✅ Available Fine-Tuned Models:")
    for model in fine_tuned_models:
        print(model)
else:
    print("❌ No fine-tuned models found.")



✅ Available Fine-Tuned Models:
ft:gpt-3.5-turbo-0125:tarangnair::AySrH3ks:ckpt-step-859
ft:gpt-3.5-turbo-0125:tarangnair::AySrH0Mi:ckpt-step-1718
ft:gpt-3.5-turbo-0125:tarangnair::AySrIowB


In [133]:
fine_tuned_model = fine_tuned_models[2]
fine_tuned_model

'ft:gpt-3.5-turbo-0125:tarangnair::AySrIowB'

In [113]:
sports_sentences = {
    "baseball": [
        "The batter swung hard and sent the ball over the fence.",
        "The pitcher struck out three batters in a row to close the inning.",
        "The stadium erupted in cheers after a walk-off home run.",
        "The team’s ace pitcher threw a no-hitter in the playoffs.",
        "The shortstop made a diving stop and turned a double play.",
        "The rookie hit a grand slam in his first major league game.",
        "The baseball manager decided to intentionally walk the best hitter.",
        "A record crowd attended the opening day of the season.",
        "The umpire called a controversial third strike to end the game.",
        "The slugger broke the single-season home run record."
    ],
    "hockey": [
        "The forward deked past two defenders and scored on the backhand.",
        "The goalie made a miraculous glove save in the final seconds.",
        "The team scored the game-winning goal in double overtime.",
        "A brutal check along the boards sent the crowd into a frenzy.",
        "The defenseman’s slap shot from the blue line found the net.",
        "The team captain hoisted the Stanley Cup in celebration.",
        "The penalty kill unit successfully defended a 5-on-3 situation.",
        "The rookie netted his first career hat trick.",
        "The enforcer dropped the gloves and fought the opposing player.",
        "The goaltender recorded a shutout in his playoff debut."
    ]
}

test_data = []

for sport in ["baseball", "hockey"]:
    for sentence in sports_sentences[sport]:
        entry = {
            "messages": [
                {"role": "system", "content": "Classify the following text into a sport category."},
                {"role": "user", "content": sentence},
                {"role": "assistant", "content": sport}
            ]
        }
        test_data.append(entry)



In [137]:
print(len(test_data))
test_data[0]

20


{'messages': [{'role': 'system',
   'content': 'Classify the following text into a sport category.'},
  {'role': 'user',
   'content': 'The batter swung hard and sent the ball over the fence.'},
  {'role': 'assistant', 'content': 'baseball'}]}

In [144]:
role_message = "Classify the following text into a sport category."
user_message = next(msg["content"] for msg in test_data[0]["messages"] if msg["role"] == "user")


response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[{"role": "system", "content": role_message},{"role": "user", "content": user_message}],
    )

print(response.choices[0].message.content.strip())

baseball


In [149]:
import time
from sklearn.metrics import accuracy_score


predictions = []
actual_labels = []


# Evaluate each test record using the fine-tuned model
for entry in test_data:
    user_message = next(msg["content"] for msg in entry["messages"] if msg["role"] == "user")
    actual_label = next(msg["content"] for msg in entry["messages"] if msg["role"] == "assistant")

    print("User input : ",user_message)
    print("Actual Label : ",actual_label)

    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[{"role": "system", "content": role_message},{"role": "user", "content": user_message}],
        max_tokens=10
    )


    predicted_label = response.choices[0].message.content.strip()

    print("Predicted Label : ",predicted_label)
    print("------------------------------------------------")

    # Store result
    predictions.append(predicted_label)
    actual_labels.append(actual_label)







User input :  The batter swung hard and sent the ball over the fence.
Actual Label :  baseball
Predicted Label :  baseball
------------------------------------------------
User input :  The pitcher struck out three batters in a row to close the inning.
Actual Label :  baseball
Predicted Label :  baseball
------------------------------------------------
User input :  The stadium erupted in cheers after a walk-off home run.
Actual Label :  baseball
Predicted Label :  baseball
------------------------------------------------
User input :  The team’s ace pitcher threw a no-hitter in the playoffs.
Actual Label :  baseball
Predicted Label :  baseball
------------------------------------------------
User input :  The shortstop made a diving stop and turned a double play.
Actual Label :  baseball
Predicted Label :  baseball
------------------------------------------------
User input :  The rookie hit a grand slam in his first major league game.
Actual Label :  baseball
Predicted Label :  baseb

In [150]:
# Calculate accuracy
accuracy = accuracy_score(actual_labels, predictions)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")


✅ Model Accuracy: 100.00%


In [156]:
# Test
response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[{"role": "system", "content": role_message},{"role": "user", "content": "The ball is hot"}],
    )


predicted_label = response.choices[0].message.content.strip()
predicted_label

'baseball'